# Confluence + LLM = QA

In [1]:
%pip install pandas --quiet
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [2]:
from os import environ
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

confluence_token = environ.get('CONFLUENCE_TOKEN')
hf_token = environ.get('HF_TOKEN')
hf_write_token = environ.get('HF_WRITE_TOKEN')
gigachat_token = environ.get('GIGACHAT_TOKEN')
gigachat_pro_token = environ.get('GIGACHAT_PRO_TOKEN')
yandex_token = environ.get('YC_API_KEY')

In [3]:
from atlassian import Confluence

confluence_url = "https://confluence.utmn.ru"
confluence = Confluence(url=confluence_url, token=confluence_token)

## Поиск документа в Confluence через CQL


 * https://atlassian-python-api.readthedocs.io/confluence.html
 * https://developer.atlassian.com/cloud/confluence/rest/v2/api-group-page/#api-pages-id-get
 * https://developer.atlassian.com/server/confluence/advanced-searching-using-cql/
 * https://spacy.io/usage/spacy-101

In [8]:
!python -m spacy download ru_core_news_sm --quiet

✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [4]:
import spacy
nlp = spacy.load("ru_core_news_sm")
needed_pos = ['NOUN', 'NUM', 'PROPN', 'ADJ', 'VERB', 'X']


def get_cql_query(spaces, question):
    exclude = ' and label != "навигация"' 
    words = [token for token in nlp(question.lower()) if not token.is_stop and
             token.pos_ in needed_pos and len(token.text) > 2]
    if len(words) == 0:
        return ()
    spaces = " or ".join([f"space = {space}" for space in spaces])
    words_with_verbs = " and ".join(list(set([f"(title ~ '{word}*' or text ~ '{word}*' or title ~ '{word.lemma_}*' or text ~ '{word.lemma_}*')"
                                              for word in words])))
    words_without_verbs = " and ".join(list(set([f"(title ~ '{word}*' or text ~ '{word}*' or title ~ '{word.lemma_}*' or text ~ '{word.lemma_}*')"
                                                 for word in words if word.pos_ != 'VERB'])))
    words_without_verbs_and_adj = " and ".join(list(set([f"(title ~ '{word}*' or text ~ '{word}*' or title ~ '{word.lemma_}*' or text ~ '{word.lemma_}*')"
                                                         for word in words if word.pos_ not in ['VERB', 'ADJ']])))
    return ("(" + spaces + ") and (" + words_with_verbs + ")" + exclude,
            "(" + spaces + ") and (" + words_without_verbs + ")" + exclude,
            "(" + spaces + ") and (" + words_without_verbs_and_adj + ")" + exclude)
    

def get_document_id(question: str) -> str:
    cql_query = get_cql_query(spaces=["study"], question=question)
    if len(cql_query) == 0:
        return "0"
    results = confluence.cql(cql_query[0], start=0, limit=1)['results']
    if len(results) == 0:
        results = confluence.cql(cql_query[1], start=0, limit=1)['results']
        if len(results) == 0:
            results = confluence.cql(cql_query[2], start=0, limit=1)['results']
            if len(results) == 0:
                return "0"

    return results[0]['content']['id']

In [4]:
from bs4 import BeautifulSoup
from langchain_community.document_loaders import PyPDFLoader

def get_document_content_by_id(page_id: str):
    children = confluence.cql(f"parent={page_id}")["results"]
    if len(children) > 0:
        return
    page = confluence.get_page_by_id(page_id, expand='space,body.export_view')
    
    page_body = page['body']['export_view']['value']
    page_download = page['_links']['base'] + page['_links']['download'] if 'download' in page['_links'].keys() else ''

    try:
        if len(page_body) > 50:
            page_body = page['body']['export_view']['value']
            soup = BeautifulSoup(page_body, 'html.parser')
            page_body_text = soup.get_text(separator=' ')
            content = page_body_text.replace(" \n ", "")
        elif '.pdf' in page_download.lower():
            loader = PyPDFLoader(page_download.split('?')[0])
            content = " ".join([page.page_content for page in loader.load_and_split()])
        else:
            return None
    except:
        return None

    return content


def get_document_content(question: str):
    page_id = get_document_id(question)
    if page_id == "0":
        return None
    return get_document_content_by_id(page_id)

In [6]:
get_document_content("Спорт?")

'Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке "Выбор модулей".\xa0 Вам нужно будет выбрать 2 интересующие Вас спортивные секции, которые будут проходить каждую неделю в одно и то же время.\xa0 Ограничения: 1) Записаться можно не более чем на 2 занятия в неделю 2) Нельзя записываться на два занятия подряд.\xa0 Вас могут не допустить на занятие, если Вы были на предыдущей паре и/или уже посетили два занятия за неделю. При этом расписание на наличие конфликтов Вы проверяете самостоятельно в соответствии с Вашим расписанием в ИС Модеус и расписанием спортивных секций (во вложенных файлах). Ваш выбор пролонгируется до конца семестра, однако в любой момент Вы можете его изменить, отписавшись от одной секции и записавшись на другую. ВАЖНО!  Студент, пропустивший два занятия подряд, будет отписан автоматически. Выбор Физической культуры откроется 06.09.2023 и будет открыт до конца семестра.\xa0 Для успешной аттестации по дисциплине «Физическая культура:

## Собственный индекс Confluence

### Структура БД

In [4]:
from sqlalchemy import create_engine
engine = create_engine(f"postgresql://{environ.get('POSTGRES_USER')}:{environ.get('POSTGRES_PASSWORD')}@{environ.get('POSTGRES_HOST')}/{environ.get('POSTGRES_DB')}", echo=False)

In [5]:
from typing import Optional
from pgvector.sqlalchemy import Vector
from sqlalchemy import Text
from sqlalchemy.orm import DeclarativeBase, Mapped, Session, mapped_column

class Base(DeclarativeBase):
    pass

class Chunk(Base):
    __tablename__ = "chunk"
    id: Mapped[int] = mapped_column(primary_key=True)
    confluence_id: Mapped[int] = mapped_column(index=True)
    text: Mapped[str] = mapped_column(Text())
    text_lem: Mapped[str] = mapped_column(Text())
    tfidf: Mapped[Optional[Vector]] = mapped_column(Vector(2777))
    doc2vec: Mapped[Optional[Vector]] = mapped_column(Vector(150))
    rubert: Mapped[Optional[Vector]] = mapped_column(Vector(312))
    rusbert: Mapped[Optional[Vector]] = mapped_column(Vector(312))
    rusbert_finetuned: Mapped[Optional[Vector]] = mapped_column(Vector(312))
    gigachat_embeddings: Mapped[Optional[Vector]] = mapped_column(Vector(1024))

In [7]:
def lower_stopword_lemmatize(text):
    return " ".join([token.lemma_ for token in nlp(str(text).lower()) if not token.is_stop and token.pos_ != 'PUNCT'])

### Выгрузка, предобработка и сохранение документов из пространства

In [8]:
page_ids = []
count_start = 0
limit = 100
pages = confluence.cql(f"space = study order by id", start=count_start, limit=limit)["results"]
while len(pages) != 0:
    page_ids = page_ids + [page['content']['id'] for page in pages if 'content' in page.keys()]
    count_start += limit
    pages = confluence.cql(f"space = study order by id", start=count_start, limit=limit)["results"]
len(page_ids)

45

In [9]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain_core.documents import Document

documents = []
for page_id in page_ids:
    page_content = get_document_content_by_id(page_id)
    if page_content is None:
        continue
    documents.append(Document(
        page_content=page_content, metadata={"page_id": page_id}
    ))
documents[:3]

[Document(page_content='Сформировать справку-вызов Вы можете самостоятельно в личном кабинете на портале "Вместе"  https://vmeste.utmn.ru/login  (раздел "Обучение"=>"Справки"). После подписания скан-копия справки-вызов направляется на корпоративную студенческую почту. ВАЖНО! На основании ст. 173 Трудового кодекса РФ и п. 4.5. Положения о текущем контроле успеваемости и промежуточной аттестации обучающихся ФГАОУ ВО ТюмГУ справка-вызов на текущую сессию студентам заочной формы обучения оформляется только при успешной сдаче предыдущей сессии и предоставляется до начала сессии. \xa0 При наличии академической задолженности за предыдущую сессию, справка-вызов не предоставляется.', metadata={'page_id': '86478987'}),
 Document(page_content='По вопросу получения справки для военкомата Вам необходимо обратиться в Отдел мобилизационной подготовки по адресу: ул. Перекопская, 15А каб.408\xa0 Телефон: (3452) 59-75-83', metadata={'page_id': '86478990'}),
 Document(page_content='Студенты очной формы о

In [10]:
text_splitter = SentenceTransformersTokenTextSplitter(model_name="saved_models/rubert-tiny2-wikiutmn")   
all_splits = text_splitter.split_documents(documents)
len(all_splits)

54

In [11]:
from sqlalchemy import text

with Session(engine) as session:
    session.execute(text('CREATE EXTENSION IF NOT EXISTS vector'))
    session.commit()
Base.metadata.create_all(engine)

In [14]:
with Session(engine) as session:
    for split in all_splits:
        doc = Chunk(
            confluence_id=int(split.metadata["page_id"]),
            text=split.page_content, 
            text_lem="" # lower_stopword_lemmatize(split.page_content)
        )
        session.add(doc)
    session.commit()   

### Эмбеддинги

In [16]:
from sqlalchemy import select

with Session(engine) as session:
   db_documents = pd.DataFrame([{"id": doc.id, 
                                     "text": doc.text, 
                                     "text_lem": doc.text_lem} for doc in session.scalars(select(Chunk).order_by(Chunk.id)).all()])
db_documents_lem = db_documents.text_lem
db_documents.head()

,id,text,text_lem
0,2,Сформировать справку - вызов Вы можете самосто...,
1,3,По вопросу получения справки для военкомата Ва...,
2,4,Студенты очной формы обучения оформляют справк...,
3,5,Для оформления академической справки ( справки...,
4,6,"По вопросу получения справки о доходах, размер...",


#### TFIDF

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidfX = tfidf_vectorizer.fit_transform(db_documents_lem)
len(tfidf_vectorizer.transform([db_documents_lem[0]]).toarray()[0])

2777

In [11]:
with Session(engine) as session:
   documents = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for doc in documents:
      doc.tfidf = tfidf_vectorizer.transform([doc.text_lem]).toarray()[0]
      session.add(doc)
      session.flush()
   session.commit()

In [12]:
def answer_tfidf(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.tfidf.cosine_distance(
                            tfidf_vectorizer.transform([lower_stopword_lemmatize(question)]).toarray()[0]
                            )).limit(1)).first().text

In [13]:
answer_tfidf("Как поменять физкультуру?")

'Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе « Отзывус » : https : / / electives. utmn. ru. Там же можно написать о своём опыте изучения элективных дисциплин. Информация, о том, как поменять электив, отправляется на корпоративную почту в первую учебную неделю семестра.'

#### BM25

In [16]:
%pip install rank_bm25 --quiet

Note: you may need to restart the kernel to use updated packages.


In [31]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split(" ") for doc in db_documents_lem]
bm25 = BM25Okapi(tokenized_corpus)
bm25

In [47]:
import numpy as np

def answer_bm25(question):
    id = np.argmax(bm25.get_scores(lower_stopword_lemmatize(question).split(" ")))
    return db_documents.text[id]
        

In [58]:
answer_bm25("Что делать, если потерял студенческий билет?")

'Для восстановления студенческого билета Вам необходимо подать заявление в Едином деканате ( ул. Семакова, д. 18, холл 3 этажа ). При себе иметь фото 3 * 4 и паспорт. Чтобы продлить студенческий билет, необходимо подойти в Единый деканат к специалисту, курирующему Ваш институт ( ул. Семакова, д. 18, каб. 302 либо 305 ). Если Вы потеряли магнитную карту ( пропуск, проходка ), необходимо подать заявление на ее восстановление в Едином деканате ( ул. Семакова, д. 18, 3 этаж ). При себе иметь паспорт или студенческий билет. Если Ваша магнитная карта ( пропуск, проходка ) не работает, необходимо обратиться в Центр информационных технологий по адресу : ул. Ленина, д. 23, каб. 310. При себе иметь карту, паспорт либо продленый студенческий билет. Телефон Единого деканата : 8 ( 3452 ) 59 - 75 - 95, email : ed @ utmn. ru'

#### Doc2vec

In [29]:
%pip install gensim --quiet

Note: you may need to restart the kernel to use updated packages.


In [14]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate([text.split() for text in db_documents_lem])]
doc2vec_model = Doc2Vec(documents, vector_size=150, window=5, min_count=1, workers=4)

len(doc2vec_model.infer_vector("мама мыла раму".split()))

150

In [29]:
with Session(engine) as session:
   documents = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for doc in documents:
      doc.doc2vec = doc2vec_model.infer_vector(doc.text_lem.split())
      session.add(doc)
      session.flush()
   session.commit()

In [15]:
def answer_doc2vec(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.doc2vec.cosine_distance(
                           doc2vec_model.infer_vector(lower_stopword_lemmatize(question).split())
                            )).limit(1)).first().text

In [16]:
answer_doc2vec("Как поменять физкультуру?")

'Заявление подается через личный кабинет на портале " Вместе ". Отпуск по уходу за ребенком до достижения им возраста трех лет предоставляется обучающемуся, являющемуся матерью ( отцом, опекуном ) на основании личного заявления, к которому прилагается копия свидетельства о рождении ребенка. Отпуск по уходу за ребенком до достижения им возраста трех лет оформляется сроком не более трех лет с момента рождения ребенка и может быть использован полностью или по частям.'

#### RuBERT-Tiny

In [17]:
import torch
from transformers import AutoTokenizer, AutoModel
rubert_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
rubert_model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
rubert_model.cpu()
# rubert_model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

len(embed_bert_cls("мама мыла раму", rubert_model, rubert_tokenizer))

312

In [33]:
with Session(engine) as session:
   documents = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for doc in documents:
      doc.rubert = embed_bert_cls(doc.text, rubert_model, rubert_tokenizer)
      session.add(doc)
      session.flush()
   session.commit()

In [18]:
def answer_rubert(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.rubert.cosine_distance(
                            embed_bert_cls(question, rubert_model, rubert_tokenizer)
                            )).limit(1)).first().text

In [19]:
answer_rubert("Как поменять физкультуру?")

'Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе « Отзывус » : https : / / electives. utmn. ru. Там же можно написать о своём опыте изучения элективных дисциплин. Информация, о том, как поменять электив, отправляется на корпоративную почту в первую учебную неделю семестра.'

#### RuSBERT-Tiny

In [20]:
from sentence_transformers import SentenceTransformer

rusbert_model = SentenceTransformer('cointegrated/rubert-tiny2', device="cpu")
len(rusbert_model.encode("мама мыла раму"))

312

In [37]:
with Session(engine) as session:
   documents = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for doc in documents:
      doc.rusbert = rusbert_model.encode(doc.text)
      session.add(doc)
      session.flush()
   session.commit()

In [21]:
def answer_rusbert(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.rusbert.cosine_distance(
                            rusbert_model.encode(question)
                            )).limit(1)).first().text

In [22]:
answer_rusbert("Как поменять физкультуру?")

'Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке " Выбор модулей ". Вам нужно будет выбрать 2 интересующие Вас спортивные секции, которые будут проходить каждую неделю в одно и то же время. Ограничения : 1 ) Записаться можно не более чем на 2 занятия в неделю 2 ) Нельзя записываться на два занятия подряд. Вас могут не допустить на занятие, если Вы были на предыдущей паре и / или уже посетили два занятия за неделю. При этом расписание на наличие конфликтов Вы проверяете самостоятельно в соответствии с Вашим расписанием в ИС Модеус и расписанием спортивных секций ( во вложенных файлах ). Ваш выбор пролонгируется до конца семестра, однако в любой момент Вы можете его изменить, отписавшись от одной секции и записавшись на другую. ВАЖНО! Студент, пропустивший два занятия подряд, будет отписан автоматически. Выбор Физической культуры откроется 06. 09. 2023 и будет открыт до конца семестра. Для успешной аттестации по дисциплине « Физическая культура : эле

#### GigaChatEmbeddings

In [7]:
%pip install --upgrade --quiet  gigachain

Note: you may need to restart the kernel to use updated packages.


In [23]:
from langchain_community.embeddings import GigaChatEmbeddings

gigachat_embeddings = GigaChatEmbeddings(credentials=gigachat_token, verify_ssl_certs=False)
result = gigachat_embeddings.embed_documents(texts=["Привет!"])
print(len(result[0]))

1024


In [ ]:
with Session(engine) as session:
   documents = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for doc in documents:
      try:
         doc.gigachat_embeddings = gigachat_embeddings.embed_documents(texts=[doc.text])[0]
      except:
         print(doc.text)
         doc.gigachat_embeddings = None
      session.add(doc)
      session.flush()
   session.commit()

In [26]:
def answer_gigachat_embeddings(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.gigachat_embeddings.cosine_distance(
                            gigachat_embeddings.embed_documents(texts=[question])[0]
                            )).limit(1)).first().text

In [29]:
answer_gigachat_embeddings("Как поменять физкультуру?")

'Прежде, чем выбрать элективы, рекомендуем почитать отзывы на сервисе « Отзывус » : https : / / electives. utmn. ru. Там же можно написать о своём опыте изучения элективных дисциплин. Информация, о том, как поменять электив, отправляется на корпоративную почту в первую учебную неделю семестра.'

## Выбор нужного фрагмента через векторный индекс

### Тестовая выборка с вопросами

In [51]:
study_questions = pd.read_csv("study_questions.csv", index_col=0)
study_questions

,question,page_id,document
0,Почему не могу получить справку-вызов?,86478987,Сформировать справку - вызов Вы можете самосто...
1,Как получить справку-вызов?,86478987,Сформировать справку - вызов Вы можете самосто...
2,Где получить справку-вызов?,86478987,Сформировать справку - вызов Вы можете самосто...
3,Где взять справку для военкомата?,86478990,Вам необходимо обратиться в Отдел мобилизацион...
4,Как получить отсрочку от армии?,86478990,Вам необходимо обратиться в Отдел мобилизацион...
...,...,...,...
106,Могу ли я в последний момент отказаться от сме...,86479065,"Прежде, чем выбрать элективы, рекомендуем почи..."
107,где почитать отзывы на элективы?,86479065,"Прежде, чем выбрать элективы, рекомендуем почи..."
108,когда можно поменять элективы?,86479065,"Прежде, чем выбрать элективы, рекомендуем почи..."
109,как выбрать электив?,86479065,"Прежде, чем выбрать элективы, рекомендуем почи..."


In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(study_questions.question, study_questions.document, test_size=0.6, random_state=666, stratify=study_questions.document)
test_questions = pd.DataFrame({"question": X_test, "document": y_test}).reset_index(drop=True)
test_questions.head()

,question,document
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...
1,Даты подачи заявления для восстановления на очку,Заявления на восстановление в Университет по о...
2,Что делать при потере проходки?,Для восстановления студенческого билета Вам не...
3,Я хожу в фитнес-клуб. Как заменить физкультуру?,Выбор спортивных секций по Физической культуре...
4,Когда мне выдадут студенческий билет после пер...,"1. Подать заявление о переводе можно лично, об..."


In [53]:
train_questions = pd.DataFrame({"question": X_train, "document": y_train}).reset_index(drop=True)
train_questions.head()

,question,document
0,Где продлять студак?,Для восстановления студенческого билета Вам не...
1,Когда можно получить справку о стипендии?,"По вопросу получения справки о доходах, размер..."
2,Как перевестись на другое направление на заочке?,Заявления о переводе принимаются два раза в го...
3,Как перевестись на другое направление?,Заявления о переводе принимаются два раза в го...
4,Могу ли я в последний момент отказаться от сме...,"Прежде, чем выбрать элективы, рекомендуем почи..."


In [54]:
# test_questions["CQL"] = test_questions["question"].apply(get_document_content)
# test_questions["tfidf"] = test_questions["question"].apply(answer_tfidf)
# test_questions["doc2vec"] = test_questions["question"].apply(answer_doc2vec)
# test_questions["rubert"] = test_questions["question"].apply(answer_rubert)
# test_questions["rusbert"] = test_questions["question"].apply(answer_rusbert)
# test_questions["gigachatembeddings"] = test_questions["question"].apply(answer_gigachat_embeddings)
test_questions["bm25"] = test_questions["question"].apply(answer_bm25)
test_questions

,question,document,bm25
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...
1,Даты подачи заявления для восстановления на очку,Заявления на восстановление в Университет по о...,Заявление на академический отпуск подается чер...
2,Что делать при потере проходки?,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...
3,Я хожу в фитнес-клуб. Как заменить физкультуру?,Выбор спортивных секций по Физической культуре...,Выбор спортивных секций по Физической культуре...
4,Когда мне выдадут студенческий билет после пер...,"1. Подать заявление о переводе можно лично, об...",Подать заявление на отчисление переводом можно...
...,...,...,...
62,как закрыть физкультуру?,Выбор спортивных секций по Физической культуре...,Выбор спортивных секций по Физической культуре...
63,когда можно поменять элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
64,как получить справку о месте учёбы?,Студенты очной формы обучения оформляют справк...,программа / специализация _ _ _ _ _ _ _ _ _ _ ...
65,Как можно получать баллы за физру?,Выбор спортивных секций по Физической культуре...,Об утверждении Регламента проведения промежуто...


### Выбор лучшего алгоритма

#### Accuracy

In [40]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

text_splitter = SentenceTransformersTokenTextSplitter(model_name="saved_models/rubert-tiny2-wikiutmn")   
test_questions["CQL"] = test_questions["CQL"].apply(lambda x: text_splitter.split_text(x)[0] if x is not None else None)
test_questions

,question,document,CQL,tfidf,doc2vec,rubert,rusbert,gigachatembeddings
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...,Заявление подается через личный кабинет на пор...,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...
1,Даты подачи заявления для восстановления на очку,Заявления на восстановление в Университет по о...,None,При равном числе голосов председатель аттестац...,"1. Подать заявление о переводе можно лично, об...",Заявление подается через личный кабинет на пор...,"При восстановлении на договорное место, после ...",Заявления на восстановление в Университет по о...
2,Что делать при потере проходки?,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...,Заявление подается через личный кабинет на пор...,Для рассмотрения возможности продления академи...,Вам необходимо обратиться в Отдел мобилизацион...,Для восстановления студенческого билета Вам не...
3,Я хожу в фитнес-клуб. Как заменить физкультуру?,Выбор спортивных секций по Физической культуре...,None,Выбор спортивных секций по Физической культуре...,Заявление подается через личный кабинет на пор...,"Прежде, чем выбрать элективы, рекомендуем почи...",Выбор спортивных секций по Физической культуре...,"Прежде, чем выбрать элективы, рекомендуем почи..."
4,Когда мне выдадут студенческий билет после пер...,"1. Подать заявление о переводе можно лично, об...",Документ подписан простой электронной подписью...,Подать заявление на отчисление переводом можно...,Заявление подается через личный кабинет на пор...,Заявление на выход из отпуска подается не позд...,"При восстановлении на договорное место, после ...",Подать заявление на отчисление переводом можно...
...,...,...,...,...,...,...,...,...
62,как закрыть физкультуру?,Выбор спортивных секций по Физической культуре...,Выбор спортивных секций по Физической культуре...,Выбор спортивных секций по Физической культуре...,Заявление подается через личный кабинет на пор...,Пакет документов для восстановления вам необхо...,Вам необходимо обратиться в Отдел мобилизацион...,Вам необходимо обратиться в Отдел мобилизацион...
63,когда можно поменять элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...",Заявление подается через личный кабинет на пор...,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
64,как получить справку о месте учёбы?,Студенты очной формы обучения оформляют справк...,None,Для оформления академической справки ( справки...,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ( ФИО указать ...,Для рассмотрения возможности продления академи...,Подать заявление на отчисление переводом можно...,Заявление подается через личный кабинет на пор...
65,Как можно получать баллы за физру?,Выбор спортивных секций по Физической культуре...,None,Об утверждении Регламента проведения промежуто...,Об утверждении Регламента проведения промежуто...,Для рассмотрения возможности продления академи...,Сформировать справку - вызов Вы можете самосто...,Студенты очной формы обучения оформляют справк...


In [55]:
for column in test_questions.columns[1:]:
    print(column, sum(test_questions[column].apply(lambda x: "" if x is None else x) == test_questions.document) / len(test_questions.document))

document 1.0
bm25 0.23880597014925373


#### ROUGE-L

In [34]:
%pip install rouge --quiet

Note: you may need to restart the kernel to use updated packages.


In [56]:
from rouge import Rouge
rouge = Rouge()

for column in test_questions.columns[1:]:
    print(column, rouge.get_scores(test_questions[column].apply(lambda x: "-" if x is None else x), test_questions["document"], avg=True)['rouge-l'])

document {'r': 1.0, 'p': 1.0, 'f': 0.9999999950000011}
bm25 {'r': 0.588169180184679, 'p': 0.4324943744842278, 'f': 0.45521461315817596}


## SBERT Fine Tuning


 * https://www.sbert.net/docs/training/overview.html
 * https://huggingface.co/blog/how-to-train-sentence-transformers

In [6]:
from sentence_transformers import SentenceTransformer

### Генерация обучающей выборки через GigaChat

In [44]:
with Session(engine) as session:
   db_documents = [doc.text for doc in session.scalars(select(Chunk).order_by(Chunk.id)).all()]
len(db_documents)

52

In [45]:
from langchain.prompts import PromptTemplate

prompt_template = """
Сделай глубокий вдох и действуй как студент. Составь 7 разнообразных вопросов к документу в тройных кавычках. Используй разговорный стиль речи и студенческую лексику.

\"\"\"
{content}
\"\"\"

Вопросы:
"""

prompt = PromptTemplate.from_template(prompt_template)

In [46]:
from langchain.llms import GigaChat
giga = GigaChat(credentials=gigachat_token, verify_ssl_certs=False)
giga_chain = prompt | giga
giga_chain.invoke({"content": db_documents[0]}).strip().split("\n")

['1. Что такое справка-вызов?',
 '  ',
 '2. Как можно сформировать справку-вызов?',
 '  ',
 '3. Какую информацию нужно предоставить после присвоения заявке статуса "Выполнено"?',
 '  ',
 '4. Какие требования предъявляются к студентам для получения справки-вызова?',
 '  ',
 '5. Какие документы регулируют процесс оформления справки-вызова?',
 '  ',
 '6. Каковы последствия отсутствия справки-вызова для студентов с академической задолженностью?',
 '  ',
 '7. Каким образом можно связаться с университетом для получения дополнительной информации о справке-вызове?']

In [47]:
gigachat_docs = []
for doc in db_documents:
    query = {"content": doc}
    giga_questions = giga_chain.invoke(query).strip().split("\n")
    for q in giga_questions:
        q = q.strip()
        if len(q) < 5:
            continue
        gigachat_docs.append({
            "question": q[3:],
            "document": doc
        })
    print(giga_questions)

['1. Что такое справка-вызов?', '  ', '2. Как можно сформировать справку-вызов?', '  ', '3. Какую информацию нужно предоставить после присвоения заявке статуса "Выполнено"?', '  ', '4. Какие требования предъявляются к студентам для получения справки-вызова?', '  ', '5. Какие документы регулируют процесс оформления справки-вызова?', '  ', '6. Каковы последствия отсутствия справки-вызова для студентов с академической задолженностью?', '  ', '7. Каким образом можно связаться с университетом для получения дополнительной информации о справке-вызове?']
['1. "Какие документы мне нужны для обращения в Отдел мобилизационной подготовки?"', '  ', '  2. "Какие услуги предоставляет Отдел мобилизационной подготовки?"', '   ', '  3. "Какие требования предъявляются к документам, которые я должен предоставить?"', '   ', '  4. "Какие сроки рассмотрения документов в Отделе мобилизационной подготовки?"', '   ', '  5. "Какие контактные данные у Отдела мобилизационной подготовки?"', '   ', '  6. "Какие проц

Giga generation stopped with reason: blacklist


['Что-то в вашем вопросе меня смущает. Может, поговорим на другую тему?']


Giga generation stopped with reason: blacklist


['Как у нейросетевой языковой модели у меня не может быть настроения, но почему-то я совсем не хочу говорить на эту тему.']
['1. Что требуется заполнить, если обучающийся планирует перевод на место, финансируемое за счет бюджетных ассигнований федерального бюджета?', '  ', '2. Что требуется заполнить, если лицо, претендующее на перевод, ранее обучалось на месте, финансируемом за счет бюджетных ассигнований федерального бюджета?', '', '3. Что требуется заполнить, если обучающийся планирует перевод на место, финансируемое за счет бюджетных ассигнований федерального бюджета?', '', '4. Что требуется заполнить, если обучающийся планирует перевод на место, финансируемое за счет бюджетных ассигнований федерального бюджета, и общая продолжительность обучения будет превышать более чем на один учебный год срок освоения образовательной программы?', '', '5. Что требуется заполнить, если обучающийся планирует перевод на место, финансируемое за счет бюджетных ассигнований федерального бюджета, и ран

...

['Как у нейросетевой языковой модели у меня не может быть настроения, но почему-то я совсем не хочу говорить на эту тему.']
['1. Каковы формы текущего контроля?', '2. Как преподаватель информирует обучающихся о текущем контроле?', '3. Какие инструменты и технологии используются для текущего контроля?', '4. Как осуществляется текущий контроль по практике?', '5. Как формируются промежуточные и окончательные оценки по дисциплине?', '6. Как осуществляется промежуточная аттестация?', '7. Как проводятся экзамены и зачеты по факультативным дисциплинам?']
['1. Каким образом проводится замена экзаменатора?', ' ', '2. Как формируются ведомости при проведении экзамена?', ' ', '3. Какова продолжительность письменного экзамена?', ' ', '4. Как должны быть оформлены письменные и устные ответы на вопросы?', ' ', '5. Какие права имеет обучающийся во время экзамена?', ' ', '6. Как объявляются результаты экзамена?', ' ', '7. Какие критерии используются при выставлении оценок за экзамен?']


Giga generation stopped with reason: blacklist


['Как у нейросетевой языковой модели у меня не может быть настроения, но почему-то я совсем не хочу говорить на эту тему.']
['1. Что такое промежуточная аттестация?', '2. Какие формы промежуточной аттестации существуют?', '3. Как проводится первая повторная промежуточная аттестация?', '4. Как проводится вторая повторная промежуточная аттестация?', '5. Как оцениваются результаты повторной промежуточной аттестации?', '6. Как оформляются результаты промежуточной аттестации?', '7. Кто несет ответственность за правильность оформления документов о промежуточной аттестации?']
['1. Что происходит с оценками после прохождения промежуточной аттестации?', '2. Кто несет ответственность за правильное внесение информации о результатах промежуточной аттестации?', '3. Кто отвечает за внесение информации о результатах ликвидации академической разницы?', '4. Как вносятся изменения и дополнения к Положению?', '5. Какие нормативные акты регулируют изменения и дополнения к Положению?', '6. Какие документы 

Giga generation stopped with reason: blacklist


['Как у нейросетевой языковой модели у меня не может быть настроения, но почему-то я совсем не хочу говорить на эту тему.']


In [48]:
gigachat_docs = pd.DataFrame(gigachat_docs)
gigachat_docs.to_csv("gigachat_docs.csv")

### Тонкая настройка

In [5]:
gigachat_docs = pd.read_csv("gigachat_docs.csv", index_col=0).reset_index(drop=True)
gigachat_docs

,question,document
0,Что такое справка-вызов?,Сформировать справку - вызов Вы можете самосто...
1,Как можно сформировать справку-вызов?,Сформировать справку - вызов Вы можете самосто...
2,Какую информацию нужно предоставить после прис...,Сформировать справку - вызов Вы можете самосто...
3,Какие требования предъявляются к студентам для...,Сформировать справку - вызов Вы можете самосто...
4,Какие документы регулируют процесс оформления ...,Сформировать справку - вызов Вы можете самосто...
...,...,...
306,Как оформляются результаты промежуточной аттес...,"проведения промежуточной аттестации, за исключ..."
307,Кто несет ответственность за правильность офор...,"проведения промежуточной аттестации, за исключ..."
308,Что происходит с оценками после прохождения пр...,". 6. 10. Из ведомости, за исключением электрон..."
309,Кто несет ответственность за правильное внесен...,". 6. 10. Из ведомости, за исключением электрон..."


In [6]:
train_questions = pd.concat([train_questions, gigachat_docs], ignore_index=True)
train_questions

,question,document
0,Где продлять студак?,Для восстановления студенческого билета Вам не...
1,Когда можно получить справку о стипендии?,"По вопросу получения справки о доходах, размер..."
2,Как перевестись на другое направление на заочке?,Заявления о переводе принимаются два раза в го...
3,Как перевестись на другое направление?,Заявления о переводе принимаются два раза в го...
4,Могу ли я в последний момент отказаться от сме...,"Прежде, чем выбрать элективы, рекомендуем почи..."
...,...,...
350,Как оформляются результаты промежуточной аттес...,"проведения промежуточной аттестации, за исключ..."
351,Кто несет ответственность за правильность офор...,"проведения промежуточной аттестации, за исключ..."
352,Что происходит с оценками после прохождения пр...,". 6. 10. Из ведомости, за исключением электрон..."
353,Кто несет ответственность за правильное внесен...,". 6. 10. Из ведомости, за исключением электрон..."


In [8]:
import math
from sentence_transformers import InputExample, losses
from torch.utils.data import DataLoader

train_set = []
for index, row in train_questions.iterrows():
    train_set.append(InputExample(texts=[row['question'], row['document']]))
    
finetuned_model = SentenceTransformer("cointegrated/rubert-tiny2", device="cuda")

train_dataloader = DataLoader(train_set, shuffle=True, batch_size=8)
train_loss = losses.MultipleNegativesRankingLoss(finetuned_model)
# train_loss = losses.MegaBatchMarginLoss(finetuned_model)

num_epochs = 10
warmup_steps = math.ceil(len(train_set) * num_epochs * 0.1)

finetuned_model.fit(train_objectives=[(train_dataloader, train_loss)], 
                    epochs=num_epochs, 
                    warmup_steps=warmup_steps,
                    output_path="saved_models/rubert-tiny2-wikiutmn")


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

Iteration:   0%|          | 0/45 [00:00<?, ?it/s]

### Индексация

In [7]:
finetuned_model = SentenceTransformer('saved_models/rubert-tiny2-wikiutmn', device="cpu")
finetuned_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 2048, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 312, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
)

In [8]:
from sqlalchemy import select

In [20]:
with Session(engine) as session:
   documents = session.scalars(select(Chunk).order_by(Chunk.id)).all()
   for doc in documents:
      doc.rusbert_finetuned = finetuned_model.encode(doc.text)
      session.add(doc)
      session.flush()
   session.commit()

In [9]:
def answer_rusbert_finetuned(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.rusbert_finetuned.cosine_distance(
                            finetuned_model.encode(question)
                            )).limit(1)).first().text

In [10]:
answer_rusbert_finetuned("Как поменять физкультуру?")

'Выбор спортивных секций по Физической культуре будет проходить в ИС Модеус во вкладке " Выбор модулей ". Вам нужно будет выбрать 2 интересующие Вас спортивные секции, которые будут проходить каждую неделю в одно и то же время. Ограничения : 1 ) Записаться можно не более чем на 2 занятия в неделю 2 ) Нельзя записываться на два занятия подряд. Вас могут не допустить на занятие, если Вы были на предыдущей паре и / или уже посетили два занятия за неделю. При этом расписание на наличие конфликтов Вы проверяете самостоятельно в соответствии с Вашим расписанием в ИС Модеус и расписанием спортивных секций ( во вложенных файлах ). Ваш выбор пролонгируется до конца семестра, однако в любой момент Вы можете его изменить, отписавшись от одной секции и записавшись на другую. ВАЖНО! Студент, пропустивший два занятия подряд, будет отписан автоматически. Выбор Физической культуры откроется 06. 09. 2023 и будет открыт до конца семестра. Для успешной аттестации по дисциплине « Физическая культура : эле

### Метрики

In [18]:
test_questions["rusbert_finetuned"] = test_questions["question"].apply(answer_rusbert_finetuned)
test_questions

,question,document,rusbert_finetuned
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...
1,Даты подачи заявления для восстановления на очку,Заявления на восстановление в Университет по о...,Пакет документов для восстановления вам необхо...
2,Что делать при потере проходки?,Для восстановления студенческого билета Вам не...,Для восстановления студенческого билета Вам не...
3,Я хожу в фитнес-клуб. Как заменить физкультуру?,Выбор спортивных секций по Физической культуре...,Выбор спортивных секций по Физической культуре...
4,Когда мне выдадут студенческий билет после пер...,"1. Подать заявление о переводе можно лично, об...","1. Подать заявление о переводе можно лично, об..."
...,...,...,...
62,как закрыть физкультуру?,Выбор спортивных секций по Физической культуре...,Выбор спортивных секций по Физической культуре...
63,когда можно поменять элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Прежде, чем выбрать элективы, рекомендуем почи..."
64,как получить справку о месте учёбы?,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...
65,Как можно получать баллы за физру?,Выбор спортивных секций по Физической культуре...,Выбор спортивных секций по Физической культуре...


#### Accuracy

In [19]:
print("rusbert_finetuned", sum(test_questions["rusbert_finetuned"].apply(lambda x: "" if x is None else x) == test_questions.document) / len(test_questions.document))

rusbert_finetuned 0.7014925373134329


#### ROUGE-L

In [20]:
from rouge import Rouge
rouge = Rouge()

print("rusbert_finetuned", rouge.get_scores(test_questions["rusbert_finetuned"].apply(lambda x: "-" if x is None else x), test_questions["document"], avg=True)['rouge-l'])

rusbert_finetuned {'r': 0.7733489022002412, 'p': 0.7566198123094525, 'f': 0.7534038029522858}


### Save to Hub

In [21]:
finetuned_model.save_to_hub(repo_id="nizamovtimur/rubert-tiny2-wikiutmn", token=hf_write_token)

model.safetensors:   0%|          | 0.00/117M [00:00<?, ?B/s]

'https://huggingface.co/nizamovtimur/rubert-tiny2-wikiutmn/commit/740236cd3cf4e8acb90ebc9012ca6afcbf07da48'

## Большие языковые модели


https://python.langchain.com/docs/use_cases/question_answering/

In [11]:
from datasets import load_dataset, DatasetDict

dataset_qda = load_dataset("nizamovtimur/wikiutmn-study-gigachat")
if isinstance(dataset_qda, DatasetDict):
    test_dataset_qda = dataset_qda['test'].to_pandas()
else:
    test_dataset_qda = pd.DataFrame()
test_dataset_qda

,question,document,human_answer
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...,"Если Вы потеряли магнитную карту ( пропуск, пр..."
1,Даты подачи заявления для восстановления на очку,Заявления на восстановление в Университет по о...,Заявления на восстановление в Университет по о...
2,Что делать при потере проходки?,Для восстановления студенческого билета Вам не...,"Если Вы потеряли магнитную карту ( пропуск, пр..."
3,Я хожу в фитнес-клуб. Как заменить физкультуру?,Выбор спортивных секций по Физической культуре...,Для оформления посещения спортивного зала вмес...
4,Когда мне выдадут студенческий билет после пер...,"1. Подать заявление о переводе можно лично, об...",В течение пяти рабочих дней после поступления ...
...,...,...,...
62,как закрыть физкультуру?,Выбор спортивных секций по Физической культуре...,1 ) Посещать академические занятия ( 3 балла з...
63,когда можно поменять элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Информация, о том, как поменять электив, отпра..."
64,как получить справку о месте учёбы?,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...
65,Как можно получать баллы за физру?,Выбор спортивных секций по Физической культуре...,1 ) Посещать академические занятия ( 3 балла з...


In [12]:
from langchain.prompts import PromptTemplate

prompt_template = """Действуй как Вопрошалыч — виртуальный помощник студента ТюмГУ.
Используй следующий текст в тройных кавычках, чтобы кратко ответить на вопрос студента.
Не придумывай и не изменяй ссылки, адреса и телефоны. Если ответа в тексте нет, напиши "ответ не найден".

\"\"\"
{context}
\"\"\"

Вопрос студента: {question}"""

prompt = PromptTemplate.from_template(prompt_template)

In [37]:
# prompt_template_with_history = """Действуй как Вопрошалыч — виртуальный помощник студента ТюмГУ.
# Кратко ответь на вопрос студента по предоставленным текстам.
# Не придумывай и не изменяй ссылки, адреса и телефоны. Если ответа нет, напиши "ответ не найден".

# \"\"\"
# {prev_context}

# {context}
# \"\"\"

# Вопрос студента: {prev_question}
# Ответ: {prev_answer}
# Вопрос студента: {question}
# Ответ:
# """

# prompt_with_history = PromptTemplate.from_template(prompt_template)

In [13]:
def get_answer(chain, question, context=None):
        if context is None:
                context = answer_rusbert_finetuned(question)
        query = {"context": context,
                 "question": question}
        return chain.invoke(query).strip()

### Метрика: BERTScore

In [55]:
%pip install bert_score --quiet

Note: you may need to restart the kernel to use updated packages.


In [14]:
from bert_score import BERTScorer
scorer = BERTScorer(lang="ru")

### GigaChat

In [14]:
from langchain.llms import GigaChat
giga = GigaChat(credentials=gigachat_token, verify_ssl_certs=False)
giga_chain = prompt | giga
get_answer(giga_chain, "К кому обратиться по вопросам стипендии?")

'По вопросам стипендии следует обратиться в Сервисный центр бухгалтерии.'

In [32]:
%%timeit
get_answer(giga_chain, "К кому обратиться по вопросам стипендии?")

876 ms ± 8.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### GigaChat Pro

In [14]:
from langchain.llms import GigaChat
giga_pro = GigaChat(model="GigaChat-Pro", credentials=gigachat_pro_token, verify_ssl_certs=False)
giga_pro_chain = prompt | giga_pro
get_answer(giga_pro_chain, "К кому обратиться по вопросам стипендии?")

'По вопросам стипендии необходимо обратиться в Сервисный центр бухгалтерии, написав на почту 12222@utmn.ru или позвонив по номеру (3452) 59-76-76. Адрес местонахождения: г. Тюмень, ул. Кирова, д. 25/1. Часы работы: пн. - чт. 09.00 - 17.00, пт. 09.00 - 16.00, обед 12.30 - 13.15, сб., вс. - выходные дни.'

In [85]:
%%timeit
get_answer(giga_pro_chain, "К кому обратиться по вопросам стипендии?")

5.47 s ± 900 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### YandexGPT

пока изучал, 149 руб 03 коп улетело...

In [17]:
%pip install --upgrade --quiet  yandex_chain

Note: you may need to restart the kernel to use updated packages.


In [34]:
from yandex_chain import YandexLLM

yagpt = YandexLLM(folder_id="b1gvhukmt6vde5nnr73s", api_key=str(yandex_token), use_lite=False)
yagpt_chain = prompt | yagpt
get_answer(yagpt_chain, "К кому обратиться по вопросам стипендии?")

'По вопросам, связанным со стипендией, вам необходимо обратиться в Сервисный центр бухгалтерии. Вы можете написать на почту 12222 @ utmn. ru или позвонить по телефону (3452) 59-76-76.'

In [35]:
%%timeit
get_answer(yagpt_chain, "К кому обратиться по вопросам стипендии?")

The slowest run took 4.28 times longer than the fastest. This could mean that an intermediate result is being cached.
1.52 s ± 808 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 1. Тест всего пайплайна

#### GigaChat

In [42]:
gigachat_answers = []
for index, row in test_dataset_qda.iterrows():
    gigachat_answers.append(get_answer(giga_chain, row['question']))
gigachat_answers = pd.Series(gigachat_answers)
gigachat_answers.head(5)

AUTHENTICATION ERROR


0    Вам следует обратиться в Единый деканат по адр...
1                                     Ответ не найден.
2    Для восстановления проходки необходимо подать ...
3    Для замены физкультуры на посещение фитнес-клу...
4    В течение 5 (пяти) рабочих дней со дня издания...
dtype: object

In [44]:
P, R, F1 = scorer.score(list(test_dataset_qda["human_answer"]), list(gigachat_answers.apply(lambda x: '' if "вет не найден" in x else x)))
f"Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'Precision: 0.607 Recall: 0.640 F1 score: 0.621'

#### GigaChat Pro

In [86]:
gigachat_pro_answers = []
for index, row in test_dataset_qda.iterrows():
    gigachat_pro_answers.append(get_answer(giga_pro_chain, row['question']))
gigachat_pro_answers = pd.Series(gigachat_pro_answers)
gigachat_pro_answers.head(5)

0    Вам необходимо обратиться в Единый деканат по ...
1                                     Ответ не найден.
2    Если вы потеряли магнитную карту (пропуск, про...
3    Для замены практических занятий по физкультуре...
4    Студенческий билет выдадут в течение 5 рабочих...
dtype: object

In [87]:
P, R, F1 = scorer.score(list(test_dataset_qda["human_answer"]), list(gigachat_pro_answers.apply(lambda x: '' if "вет не найден" in x else x)))
f"Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'Precision: 0.680 Recall: 0.691 F1 score: 0.683'

#### YandexGPT 3

In [30]:
yagpt_answers = []
for index, row in test_dataset_qda.iterrows():
    yagpt_answers.append(get_answer(yagpt_chain, row['question']))
yagpt_answers = pd.Series(yagpt_answers)
yagpt_answers.head(5)

0    Вам нужно подать заявление на восстановление м...
1                                     Ответ не найден.
2    Необходимо подать заявление на восстановление ...
3    Для оформления посещения частного фитнес-клуба...
4    В течение 5 рабочих дней со дня издания приказ...
dtype: object

In [31]:
P, R, F1 = scorer.score(list(test_dataset_qda["human_answer"]), list(yagpt_answers.apply(lambda x: '' if "вет не найден" in x else x)))
f"Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'Precision: 0.678 Recall: 0.686 F1 score: 0.679'

### 2. Тест генерации по заранее найденному документу

#### GigaChat

In [71]:
gigachat_answers = []
for index, row in test_dataset_qda.iterrows():
    gigachat_answers.append(get_answer(giga_chain, row['question'], row['document']))
gigachat_answers = pd.Series(gigachat_answers)
gigachat_answers.head(5)

AUTHENTICATION ERROR


0    Вам следует обратиться в Единый деканат по адр...
1    Заявления на восстановление в Университет по о...
2    Для восстановления проходки необходимо подать ...
3    Для замены физкультуры необходимо обратиться к...
4    В течение 5 (пяти) рабочих дней со дня издания...
dtype: object

In [72]:
P, R, F1 = scorer.score(list(test_dataset_qda["human_answer"]), list(gigachat_answers.apply(lambda x: '' if len(x) < 18 else x)))
f"Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'Precision: 0.680 Recall: 0.729 F1 score: 0.701'

#### GigaChat Pro

In [17]:
gigachat_pro_answers = []
for index, row in test_dataset_qda[14:].iterrows():
    gigachat_pro_answers.append(get_answer(giga_pro_chain, row['question'], row['document']))
gigachat_pro_answers = pd.Series(gigachat_pro_answers)
gigachat_pro_answers.head(5)

0    Для подачи заявления на перевод в ТюмГУ из дру...
1    Об элективах можно узнать на сервисе «Отзывус»...
2    Заявление на выход из академического отпуска п...
3                                     Ответ не найден.
4    Если вам не пришла стипендия, вам следует обра...
dtype: object

In [35]:
P, R, F1 = scorer.score(list(test_dataset_qda["human_answer"]), list(gigachat_pro_answers.apply(lambda x: '' if len(x) < 18 else x)))
f"Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'Precision: 0.786 Recall: 0.789 F1 score: 0.785'

#### YandexGPT 3

тут возник такой мем: `ai.textGenerationCompletionRequestsPerHour.rate rate quota limit exceed: allowed 100 requests`

хаха, лан, пошёл гулять в магаз

In [46]:
yagpt_answers = []
for index, row in test_dataset_qda.iterrows():
    yagpt_answers.append(get_answer(yagpt_chain, row['question'], row['document']))
yagpt_answers = pd.Series(yagpt_answers)
yagpt_answers.head(5)

0    Согласно тексту, для перевода в ТюмГУ необходи...
1    Обратитесь к заключению врачебной комиссии. Ес...
2    В представленном тексте есть информация только...
3    Для того чтобы получить отпуск по уходу за реб...
4    Заявление о выходе из академического отпуска п...
dtype: object

In [68]:
P, R, F1 = scorer.score(list(test_dataset_qda["human_answer"]), list(yagpt_answers.apply(lambda x: '' if len(x) < 18 else x)))
f"Precision: {P.mean():.3f} Recall: {R.mean():.3f} F1 score: {F1.mean():.3f}"

'Precision: 0.792 Recall: 0.794 F1 score: 0.790'

### 3. Тест нахождения ответа

In [15]:
# generated by CohereForAI/c4ai-command-r-plus
random_questions = """
Какой процент студентов поступает в университет из другой страны?
Какие факторы влияют на выбор студентами своего учебного заведения?
Какие существуют методы повышения мотивации студентов во время обучения?
Какие существуют методы борьбы с академическим плагиатом?
Как развиваются онлайн-курсы и дистанционное обучение в современном университете?
Какие существуют методы повышения академического Writing навыков у студентов?
Как студентам лучше всего управлять своим временем для достижения академического успеха?
Какие существуют методы повышения активности студентов на лекциях?
Как преподаватели могут эффективно использовать технологии в обучении?
Какие существуют программы развития критического мышления у студентов?
Как студентам лучше всего готовиться к экзаменам?
Какие факторы влияют на успеваемость студентов?
Какие существуют методы предотвращения студенческого стресса и выгорания?
Как студентам лучше всего искать и использовать академические источники для своих работ?
Какие существуют программы обучения за рубежом и как они влияют на студентов?
Какой процент студентов совмещает работу и учебу?
Какие существуют методы повышения разнообразия и инклюзивности в студенческом сообществе?
Какой процент студентов продолжает обучение в аспирантуре?
Какие факторы влияют на выбор студентами своей специальности?
Какие существуют методы повышения академического честности?
Как студентам лучше всего управлять своими финансами во время обучения?
Какие существуют программы поддержки психического здоровья студентов?
Какие существуют методы эффективного обучения в группе?
Как преподаватели могут адаптировать свое обучение для разных стилей обучения студентов?
Какие существуют программы развития межкультурной компетентности?
Как студентам лучше всего искать стажировки и возможности для развития карьеры?
Какие факторы влияют на вовлеченность студентов в учебный процесс?
Какие существуют методы повышения качества обучения в университете?
Как преподаватели могут эффективно использовать активные методы обучения?
Какие существуют программы поддержки студентов из малообеспеченных семей?
Какие существуют методы предотвращения студенческого отсева?
Как студентам лучше всего развивать свои soft skills?""".split("\n")
random_questions = pd.Series(random_questions)
random_questions = pd.DataFrame({"question": random_questions, "document": random_questions.apply(answer_rusbert_finetuned), "label": 0})
actual_questions = pd.DataFrame({"question": test_dataset_qda["question"].head(33), "document": test_dataset_qda["document"].head(33), "label": 1})
question_documents_label = pd.concat([actual_questions, random_questions], ignore_index=True)
question_documents_label.sample(5)

,question,document,label
10,"Я слетел с бюджета, как мне восстановиться?","Восстановление лица, ранее обучавшегося за сче...",1
23,Сколько делается табель с оценками?,Для оформления академической справки ( справки...,1
11,как перевестись на другое направление?,Заявления о переводе принимаются два раза в го...,1
1,Даты подачи заявления для восстановления на очку,Заявления на восстановление в Университет по о...,1
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...,1


#### GigaChat

In [58]:
gigachat_answers = []
for index, row in question_documents_label.iterrows():
    gigachat_answers.append(get_answer(giga_chain, row['question'], row['document']))
gigachat_answers = pd.Series(gigachat_answers)
question_documents_label["gigachat"] = gigachat_answers.apply(lambda x: 0 if "вет не найден" in x else 1)
question_documents_label.sample(5)

AUTHENTICATION ERROR
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist


,question,document,label,gigachat
39,Как студентам лучше всего управлять своим врем...,Для оформления академической справки ( справки...,0,0
17,Даты подачи заявления для восстановления на за...,Заявления на восстановление в ТюмГУ принимаютс...,1,0
37,Как развиваются онлайн-курсы и дистанционное о...,Студенты очной формы обучения оформляют справк...,0,0
46,Как студентам лучше всего искать и использоват...,Для оформления академической справки ( справки...,0,0
30,Могу ли я восстановиться на заочке?,Заявления на восстановление в ТюмГУ принимаютс...,1,1


In [60]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(question_documents_label.label, question_documents_label.gigachat, average='weighted')

(0.8336397058823529, 0.8333333333333334, 0.8332950631458095, None)

#### GigaChat Pro

In [17]:
gigachat_pro_answers = []
for index, row in question_documents_label[42:].iterrows():
    gigachat_pro_answers.append(get_answer(giga_pro_chain, row['question'], row['document']))
gigachat_pro_answers = pd.Series(gigachat_pro_answers)

Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist


In [23]:
question_documents_label["gigachat_pro"] = gigachat_pro_answers.apply(lambda x: 0 if "вет не найден" in x else 1)
question_documents_label.sample(5)

,question,document,label,gigachat_pro
56,Какие существуют методы эффективного обучения ...,Сопровождение специализированных образовательн...,0,0
34,Какой процент студентов поступает в университе...,Документ подписан простой электронной подписью...,0,0
65,Как студентам лучше всего развивать свои soft ...,Сопровождение специализированных образовательн...,0,0
51,Какой процент студентов продолжает обучение в ...,Документ подписан простой электронной подписью...,0,0
27,Куда обращаться по поводу оплаты обучения?,"По вопросам оплаты обучения, суммы задолженнос...",1,1


In [24]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(question_documents_label.label, question_documents_label.gigachat_pro, average='weighted')

(0.8273809523809524, 0.803030303030303, 0.7992982456140351, None)

#### YandexGPT 3

In [78]:
yagpt_answers = []
for index, row in question_documents_label.iterrows():
    yagpt_answers.append(get_answer(yagpt_chain, row['question'], row['document']))
yagpt_answers = pd.Series(yagpt_answers)
question_documents_label["yagpt"] = yagpt_answers.apply(lambda x: 0 if "не найден" in x else 1)
question_documents_label.sample(5)

,question,document,label,yagpt
57,Какие существуют программы развития межкультур...,Документ подписан простой электронной подписью...,0,1
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...,1,1
26,Где получить информацию о справке в военкомат?,Вам необходимо обратиться в Отдел мобилизацион...,1,1
14,Какие доки нужны для перевода в ТюмГУ из друго...,Пакет документов для подачи заявления на перев...,1,1
38,Какие существуют методы повышения академическо...,Сопровождение специализированных образовательн...,0,1


In [82]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(question_documents_label.label, question_documents_label.yagpt, average='weighted')

(0.4678362573099415, 0.48484848484848486, 0.40634920634920635, None)